In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [16]:
# 取得暢銷榜頁面文件
url = 'https://www.books.com.tw/web/sys_saletopb/books/?attribute=30'
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"
headers = {"User-Agent" : userAgent}

resp = requests.get(url, headers = headers)
soup = BeautifulSoup(resp.text, 'html.parser')

# 取得所有的暢銷類型的連結網址
catalog = soup.find(class_='mod_b type02_l001-1 clearfix')
catalogList = catalog.find_all('a')[0:1]

catalogDict = dict()
for c in catalogList:
    title = c.text
    href = c.get('href')
    href = href.split('&')[0]
    catalogDict[title] = href
# 逐一爬取不同類型暢銷榜
for title, url in catalogDict.items():
    time.sleep(5)  # 讓爬蟲睡個覺
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')

In [19]:
df = pd.DataFrame(columns=['類型', '書名', '作者', '優惠價'])

# 逐一爬取不同類型暢銷榜
for title, url in catalogDict.items():
    time.sleep(5)  # 讓爬蟲睡個覺
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')

    # 取得書架區塊 books、取得書架上所有書籍區塊 items
    books = soup.find(class_='mod_a clearfix')
    items = books.find_all(class_='item')

    # 逐一解析書籍區塊內容
    for i, item in enumerate(items):
        if i == 20:  # 只爬 Top 20的書
            break
        time.sleep(5)

        # 取得書名
        book_name = item.find('img').get('alt')

        # 取得作者        
        if item.find_all('li')[0].text[0:2] == '作者':
            book_author = item.find_all('li')[0].find('a').text
        else:
            book_author = ""

        # 取得價格
        book_price = item.find_all('b')[-1].text

        # 存入 dataframe
        df.loc[len(df)+1] = None
        df.iloc[-1]['類型'] = title
        df.iloc[-1]['書名'] = book_name
        df.iloc[-1]['作者'] = book_author
        df.iloc[-1]['優惠價'] = book_price

In [20]:
df

,類型,書名,作者,優惠價
1,總榜,活出心中的太極,太極人,253
2,總榜,原子習慣：細微改變帶來巨大成就的實證法則,詹姆斯‧克利爾,261
3,總榜,林務局2022「假面之森」月曆,行政院農業委員會林務局,255
4,總榜,ONE PIECE航海王 100(首刷限定版),尾田榮一郎,123
5,總榜,為什麼不能想怎樣就怎樣：王宏哲給孩子的情緒教育繪本2（贈1桌遊1學具）,王宏哲,315
6,總榜,屁屁偵探讀本10：屁屁偵探戀愛了？！,Troll,237
7,總榜,內在原力：9個設定，活出最好的人生版本,愛瑞克,300
8,總榜,股息Cover我每一天：600張存股達人絕活全公開,大俠武林,253
9,總榜,正韓小菜：五星韓廚的道地韓國小菜！從開胃泡菜到麻藥雞蛋，65道零失敗偷飯料理,孫榮KaiSon,458
10,總榜,社會在走，歷史要懂：呂捷開講,呂捷,261
